In [1]:
import pandas as pd
import numpy as np

read applicants from scorecard training file and define product range

In [2]:
#applicants = pd.read_json('resources/scorecardtest.json')
applicants = pd.read_csv('resources/german_credit_data.csv')
applicants.rename(columns = {'Unnamed: 0':'Applicant'}, inplace = True)
applicants = applicants.drop(['Credit amount', 'Duration'], axis=1)

duration = pd.DataFrame({'Duration' : [4, 8, 12, 24, 36, 72]})
amount = pd.DataFrame({'Credit amount' : [250, 500, 1000, 2500, 5000, 10000, 20000]})
interest = pd.DataFrame({'Interest rate' : [0.05, 0.07, 0.10, 0.15]})

enumerate all combinations of applicants and products

In [3]:
def cartesian_product_basic(left, right): return (
       left.assign(key=1).merge(right.assign(key=1), on='key').drop('key', 1))

products = cartesian_product_basic(applicants, duration)
products = cartesian_product_basic(products, amount)
products = cartesian_product_basic(products, interest)
products.head()

,Applicant,Age,Sex,Job,Housing,Saving accounts,Checking account,Purpose,Risk,Duration,Credit amount,Interest rate
0,0,67,male,2,own,NaN,little,radio/TV,good,4,250,0.05
1,0,67,male,2,own,NaN,little,radio/TV,good,4,250,0.07
2,0,67,male,2,own,NaN,little,radio/TV,good,4,250,0.10
3,0,67,male,2,own,NaN,little,radio/TV,good,4,250,0.15
4,0,67,male,2,own,NaN,little,radio/TV,good,4,500,0.05


instantiate the scorecard and predict outcomes for all combinations

In [4]:
from classes.scorecard import Scorecard
scorecard = Scorecard()
scorecard.train()

products = pd.merge(products, 
                    pd.DataFrame(scorecard.predictFromJson(products, proba=True), 
                                 columns=['pBad', 'pGood']), 
                    left_index=True, right_index=True)

determine monthly payments and total cost of credit

In [6]:
def calculate_payments(row): return(
    (row['Interest rate'] / 12) / (1 - pow(1+(row['Interest rate']/12), -row['Duration'])) * row['Credit amount'])

products['Monthly payment'] = products.apply(calculate_payments, axis=1)
products['Total cost'] = products['Monthly payment'] * products['Duration']
products.head()

,Applicant,Age,Sex,Job,Housing,Saving accounts,Checking account,Purpose,Risk,Duration,Credit amount,Interest rate,pBad,pGood,Monthly payment,Total cost
0,0,67,male,2,own,NaN,little,radio/TV,good,4,250,0.05,0.083705,0.916295,63.152395,252.609581
1,0,67,male,2,own,NaN,little,radio/TV,good,4,250,0.07,0.083705,0.916295,63.414109,253.656436
2,0,67,male,2,own,NaN,little,radio/TV,good,4,250,0.10,0.083705,0.916295,63.807486,255.229944
3,0,67,male,2,own,NaN,little,radio/TV,good,4,250,0.15,0.083705,0.916295,64.465256,257.861023
4,0,67,male,2,own,NaN,little,radio/TV,good,4,500,0.05,0.084231,0.915769,126.304790,505.219161


determine profitability

In [7]:
def calculate_profit(row): return(
    (row['Total cost'] - row['Credit amount']) * row['pGood'] - row['Credit amount'] * row['pBad'])

products['Profit'] = products.apply(calculate_profit, axis=1)

finally filter according to chosen criteria and output

In [8]:
filtered_products = products[products['Profit'] > 0]
filtered_products = filtered_products[products['Monthly payment'] < 100]
filtered_products = filtered_products.groupby(['Applicant', 'Duration', 'Credit amount']).agg({'Interest rate' : np.min})
filtered_products

/home/daniel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Interest rate
Applicant Duration Credit amount               
5         24       250                     0.15
                   500                     0.15
                   1000                    0.15
          36       250                     0.15
                   500                     0.15
                   1000                    0.15
                   2500                    0.15
6         8        250                     0.15
                   500                     0.15
          12       250                     0.10
                   500                     0.10
                   1000                    0.10
          24       250                     0.07
                   500                     0.07
                   1000                    0.07
          36       250                     0.07
                   500                     0.07
                   1000                    0.07
                   2500                    0.07
          72       250                     0.15
                   500                     0.15
                   1000                    0.15
                   2500                    0.15
8         4        250                     0.07
          8        250                     0.05
                   500                     0.05
          12       250                     0.05
                   500                     0.05
                   1000                    0.05
          24       250                     0.05
...                                         ...
991       72       250                     0.15
                   500                     0.15
                   1000                    0.15
994       4        250                     0.15
          8        250                     0.10
                   500                     0.10
          12       250                     0.07
                   500                     0.10
                   1000                    0.10
          24       250                     0.07
                   500                     0.07
                   1000                    0.07
          36       250                     0.07
                   500                     0.07
                   1000                    0.07
                   2500                    0.07
          72       250                     0.10
                   500                     0.10
                   1000                    0.10
                   2500                    0.10
997       12       250                     0.15
                   500                     0.15
                   1000                    0.15
          24       250                     0.10
                   500                     0.10
                   1000                    0.15
          36       250                     0.10
                   500                     0.15
                   1000                    0.15
                   2500                    0.15

[3331 rows x 1 columns]